In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("demo").getOrCreate()


24/08/16 20:29:36 WARN Utils: Your hostname, manu-pc resolves to a loopback address: 127.0.1.1; using 192.168.157.41 instead (on interface wlp58s0)
24/08/16 20:29:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/16 20:29:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
data = [("anshu", 95, 4),
        ('aparna', 96, 4),
        ('roy', 93, 5),
        ('abhi', 95, 4),
        ('sam', 96, 5)

]
column = ['name', 'marks', 'standard']

df = spark.createDataFrame(data,column)
df.show()
querry  = """
with high_table as(
    select name, Rank() over ( partition by standard order by marks) as rnk
    from table
)
select name as high_scorer, marks as highest_mark, standard from table
where name in (
    select name from high_table
    where rnk = 1
)

"""
df.createOrReplaceTempView("table")
spark.sql(querry).show()

+------+-----+--------+
|  name|marks|standard|
+------+-----+--------+
| anshu|   95|       4|
|aparna|   96|       4|
|   roy|   93|       5|
|  abhi|   95|       4|
|   sam|   96|       5|
+------+-----+--------+



+-----------+------------+--------+
|high_scorer|highest_mark|standard|
+-----------+------------+--------+
|      anshu|          95|       4|
|        roy|          93|       5|
|       abhi|          95|       4|
+-----------+------------+--------+



In [37]:
from pyspark.sql.functions import expr
data = [("anshu", 95, 4),
        ('aparna', 96, 4),
        ('roy', 93, 5),
        ('abhi', 96, 4),
        ('sam', 96, 5)
]
columns = ['name', 'marks', 'standard']

df = spark.createDataFrame(data, columns)
# df.show()
df.withColumn("remark", expr("case when marks > 95 then 'great' else 'good' end")).show()



+------+-----+--------+------+
|  name|marks|standard|remark|
+------+-----+--------+------+
| anshu|   95|       4|  good|
|aparna|   96|       4| great|
|   roy|   93|       5|  good|
|  abhi|   96|       4| great|
|   sam|   96|       5| great|
+------+-----+--------+------+



In [34]:
from pyspark.sql.functions import expr

data = [("anshu", 95, 4),
        ('aparna', 96, 4),
        ('roy', 93, 5),
        ('abhi', 96, 4),
        ('sam', 96, 5)
]
columns = ['name', 'marks', 'standard']

df = spark.createDataFrame(data, columns)

# categorization using sql querry
df = df.withColumn("remark", expr("CASE WHEN marks > 95 THEN 'great' ELSE 'good' END"))
df.show()


+------+-----+--------+------+
|  name|marks|standard|remark|
+------+-----+--------+------+
| anshu|   95|       4|  good|
|aparna|   96|       4| great|
|   roy|   93|       5|  good|
|  abhi|   96|       4| great|
|   sam|   96|       5| great|
+------+-----+--------+------+



In [38]:
# Name Length count
from pyspark.sql.functions import col,split

data = [
    ("manish kumar",),
    ("nil nitin mukesh",),
    ('saurabh',)
]

df = spark.createDataFrame(data,['name'])

df = df.withColumn("name_length", split(col("name"), " "))
df = df.withColumn("f_name", split(col("name"), " ")[0])
df.show()

+----------------+--------------------+-------+
|            name|         name_length| f_name|
+----------------+--------------------+-------+
|    manish kumar|     [manish, kumar]| manish|
|nil nitin mukesh|[nil, nitin, mukesh]|    nil|
|         saurabh|           [saurabh]|saurabh|
+----------------+--------------------+-------+



In [22]:
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- name_length: array (nullable = true)
 |    |-- element: string (containsNull = false)



In [27]:
from pyspark.sql.functions import size
df.withColumn("name_length", size(split(col("name"), " "))).show()


+----------------+-----------+
|            name|name_length|
+----------------+-----------+
|    manish kumar|          2|
|nil nitin mukesh|          3|
|         saurabh|          1|
+----------------+-----------+

